In [ ]:
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag


In [ ]:
plate_positions = {}
with open('plate_positions.txt', 'r') as f:
    for line in f:
        frame_str, coords_str = line.split(': ')
        frame_number = int(frame_str)
        numbers = re.findall(r'\d+', coords_str)
        box = np.array(list(map(int, numbers))).reshape(4, 2)
        plate_positions[frame_number] = box

measurements = {frame: box.flatten() for frame, box in plate_positions.items()}

In [ ]:

dim_x = 16  
dim_z = 8   

kf = KalmanFilter(dim_x=dim_x, dim_z=dim_z)

dt = 1.0  
kf.F = np.eye(dim_x)
for i in range(dim_z):
    kf.F[i, i + dim_z] = dt

kf.H = np.zeros((dim_z, dim_x))
for i in range(dim_z):
    kf.H[i, i] = 1

kf.R *= 5  

q_val = 0.1  # Process noise variance
q_block = Q_discrete_white_noise(dim=2, dt=dt, var=q_val)

q_list = [q_block] * (dim_x // 2)
kf.Q = block_diag(*q_list)


first_frame = sorted(plate_positions.keys())[0]
initial_measurement = plate_positions[first_frame].flatten()
kf.x = np.zeros(dim_x)
kf.x[:dim_z] = initial_measurement

kf.P *= 100.

last_frame = max(plate_positions.keys())
filtered_states = []
measurements_list = []

for frame_num in range(last_frame + 1):
    kf.predict()
    
    if frame_num in measurements:
        z = measurements[frame_num]
        kf.update(z)
        measurements_list.append(z)
    else:
        measurements_list.append(np.full(dim_z, np.nan))
    
    filtered_states.append(kf.x[:dim_z].copy())

filtered_states = np.array(filtered_states)
measurements = np.array(measurements_list)


In [ ]:
time = np.arange(len(filtered_states))
measurements_arr = np.array(measurements)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

for i in range(4):
    ax1.plot(time, filtered_states[:, i*2], label=f'Filtered X{i+1}')
    ax1.plot(time, measurements_arr[:, i*2], linestyle='--', label=f'Measured X{i+1}', alpha=0.7)

ax1.set_title('Filtered vs. Measured X Coordinates Over Time')
ax1.set_ylabel('X Position')
ax1.legend()
ax1.grid(True)

for i in range(4):
    ax2.plot(time, filtered_states[:, i*2 + 1], label=f'Filtered Y{i+1}')
    ax2.plot(time, measurements_arr[:, i*2 + 1], linestyle='--', label=f'Measured Y{i+1}', alpha=0.7)

ax2.set_title('Filtered vs. Measured Y Coordinates Over Time')
ax2.set_xlabel('Time (frame)')
ax2.set_ylabel('Y Position')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:

PLATE_WIDTH = 0.520
PLATE_HEIGHT = 0.110

FPS = 30

object_points = np.array([
    [-PLATE_WIDTH / 2, -PLATE_HEIGHT / 2, 0],  # Bottom-left
    [ PLATE_WIDTH / 2, -PLATE_HEIGHT / 2, 0],  # Bottom-right
    [ PLATE_WIDTH / 2,  PLATE_HEIGHT / 2, 0],  # Top-right
    [-PLATE_WIDTH / 2,  PLATE_HEIGHT / 2, 0]   # Top-left
], dtype=np.float32)

image_width = 1080
image_height = 1920

focal_length_mm = 24
sensor_width_mm = 5.4
focal_length = (focal_length_mm / sensor_width_mm) * image_width

camera_center_x = image_width / 2
camera_center_y = image_height / 2

camera_matrix = np.array([
    [focal_length, 0, camera_center_x],
    [0, focal_length, camera_center_y],
    [0, 0, 1]
], dtype=np.float32)

dist_coeffs = np.array([-0.15, 0.0, 0, 0], dtype=np.float32)


In [ ]:

distances = []
for i in range(len(filtered_states)):
    image_points = filtered_states[i].reshape((4, 2)).astype(np.float32)
    if np.isnan(image_points).any():
        distances.append(np.nan)
        continue

    success, rvec, tvec = cv2.solvePnP(object_points, image_points, camera_matrix, dist_coeffs)

    if success:
        distance = tvec[2][0]
        distances.append(distance)
    else:
        distances.append(np.nan)

distances = pd.Series(distances)


In [ ]:

delta_distance = distances.diff()

speed_mps = np.abs(delta_distance * FPS)
speed_kmh_raw = speed_mps * 3.6

MAX_SPEED_KMH = 250
speed_kmh = speed_kmh_raw.copy()
speed_kmh[speed_kmh > MAX_SPEED_KMH] = np.nan


In [ ]:

window_size = 10
smoothed_speed_kmh = speed_kmh.rolling(window=window_size, min_periods=1).mean()


In [ ]:

plt.figure(figsize=(15, 7))

# plt.plot(time, speed_kmh, label='Raw Speed (km/h)', alpha=0.5, linestyle='--')
plt.plot(time, smoothed_speed_kmh, label=f'Smoothed Speed (km/h, window={window_size})', color='red', linewidth=2)

plt.title('Vehicle Speed Over Time')
plt.xlabel('Time (frame)')
plt.ylabel('Speed (km/h)')
plt.legend()
plt.grid(True)
plt.show()
